# Visualise correlation wavefield

This little notebook is intended to provide an easy visualisation of the noise correlation wavefield, produced by the 'Correlation_Analysis', the output of which it requires as input.

# 0. Python packages

In [1]:
import matplotlib.pyplot as plt
from pandas import read_csv
import numpy as np
import mplleaflet
from geographiclib.geodesic import Geodesic
geod = Geodesic.WGS84

# 1. Input

The main required input are the raw and the f-k filtered correlation wavefields (forward and reverse). The latter are the output of the 'Correlation_Analysis' notebook.

In [10]:
# Time increment [s].
dt=0.0050124999999999753

# Read correlation wavefields.
cct=np.load('cct_raw.npy')
cct_fwd=np.load('cct_forward.npy')
cct_rev=np.load('cct_reverse.npy')

# Number of channels to use.
nc_max=100

# Read in the GPS locations from file
section_location_all = read_csv('/Users/andreas/Codes/BernCorrelations/section_location_all.csv')

# 2. Initialisation

Truncate (if needed) the correlation wavefield (number of channels) and apply normalisations for better plotting.

In [11]:
# Restrict the number of channels to use.
cct=cct[:nc_max,:]
cct_fwd=cct_fwd[:nc_max,:]
cct_rev=cct_rev[:nc_max,:]

# Number of channels and time steps.
nc=np.shape(cct)[0]
nt=np.shape(cct)[1]

# Modify correlation wavefields for plotting (absolute value and normalisation).
cct=cct/np.max(np.abs(cct))
n=np.max([np.abs(cct_fwd),np.abs(cct_rev)])
cct_fwd=np.abs(cct_fwd)/n
cct_rev=np.abs(cct_rev)/n

# 3. Channel locations

In [12]:
# Plot map figure with Bern coordinates for interpolated channels (where possible to clearly determine)
%matplotlib inline
plt.figure(figsize=(10,10))
plt.scatter(section_location_all['longitude'],section_location_all['latitude'],s=10,c='b')
mplleaflet.display()

In [13]:
# Put channel locations into a dictionary for easier access. Contains channel number; latitude; longitude
sec_loc_dict = dict()
for i in section_location_all.iterrows(): sec_loc_dict.update({f"{int(i[1][0])}":[i[1][1],i[1][2]]})

In [14]:
# Number of starting channel.
channel_start=950 

# Fill arrays of latitudes and longitudes of channels.
lats=np.zeros(nc)
lons=np.zeros(nc)

i=0
k=0
while k < nc:
    if f'{channel_start+i}' in sec_loc_dict.keys():
        lats[k]=sec_loc_dict[f'{channel_start+i}'][0]
        lons[k]=sec_loc_dict[f'{channel_start+i}'][1]
        k+=1
    i+=1

# 4. Plot wavefield

We plot the wavefield for a selected time step.

In [22]:
# Time of snapshot [s].
t=0.1
# Time index.
it=int(np.round(t/dt))

## 4.1. Raw correlation wavefield

In [23]:
plt.figure(figsize=(10,10))

for i in range(nc):
    if cct[i,it]<0.0:
        r=1.0
        g=(1.0+cct[i,it])**0.5
        b=(1.0+cct[i,it])**0.5
    else:
        r=1.0-cct[i,it]
        g=1.0-cct[i,it]
        b=1.0
    color=[r,g,b]
    plt.scatter(lons[i],lats[i],s=200,color=color)

mplleaflet.display()

## 4.2. Forward and reverse correlation wavefield

In [24]:
# Offsets between forward abd reverse part.
dlat=-4.0*(lats[1]-lats[0])
dlon=2.0*(lons[1]-lons[0])

# Normalisation.
n=np.max([cct_fwd[:,it],cct_rev[:,it]])

# Plot wavefields.
plt.figure(figsize=(10,10))

# Forward propagating part.
for i in range(nc):
    r=(1.0-cct_fwd[i,it]/n)**0.5
    g=(1.0-cct_fwd[i,it]/n)**0.5
    b=1.0
    color=[r,g,b]
    plt.scatter(lons[i],lats[i],s=150,color=color)

# Reverse propagating part.
for i in range(nc):
    r=1.0
    g=(1.0-cct_rev[i,it]/n)**0.5
    b=(1.0-cct_rev[i,it]/n)**0.5
    color=[r,g,b]
    plt.scatter(lons[i]+dlon,lats[i]+dlat,s=150,color=color)
    
filename='snapshot_'+str(it)+'.html'
mplleaflet.display()